In [1]:
from pulp import *
import utils

In [43]:
instance_number = 5
file_path = f"../instances/ins-{instance_number}.txt"
w, n, dims = utils.read_output(file_path)

In [44]:
WIDTH = 0
HEIGHT = 1

In [45]:
upper_bound = sum(dims[:, HEIGHT])
k = 4 # Number of constraints

In [46]:
prob = LpProblem("VLSI", LpMinimize)

In [47]:
L = LpVariable("L", 0, upper_bound, LpInteger)
pos_x = LpVariable.dicts("pos_x", range(n), 0, w, LpInteger)
pos_y = LpVariable.dicts("pos_y", range(n), 0, upper_bound, LpInteger)

In [48]:
M0 = M1 = w
M2 = M3 = upper_bound

In [49]:
pos_chosen = LpVariable.dict("pos_chosen", (range(n), range(n), range(k)), 0, 1, LpInteger)

In [50]:
prob += L

In [51]:
for i in range(n):
    prob += L >= pos_y[i] + dims[i, HEIGHT]
    for j in range(i + 1, n):
        prob += pos_x[i] + dims[i, WIDTH] <= pos_x[j] + M0 * pos_chosen[i, j, 0]
        prob += pos_x[j] + dims[j, WIDTH] <= pos_x[i] + M1 * pos_chosen[i, j, 1]
        prob += pos_y[i] + dims[i, HEIGHT] <= pos_y[j] + M2 * pos_chosen[i, j, 2]
        prob += pos_y[j] + dims[j, HEIGHT] <= pos_y[i] + M3 * pos_chosen[i, j, 3]
        prob += lpSum([pos_chosen[i, j, k] for k in range(k)]) <= 3

        

In [52]:
status = prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/olestole/code/unibo/combinatorial/VLSIdesign/venv/lib/python3.7/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/vn/xw7vy4yj4p59wc8lpy7gnh200000gn/T/63b3c94abd804e5f86a9e907f72351d1-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/vn/xw7vy4yj4p59wc8lpy7gnh200000gn/T/63b3c94abd804e5f86a9e907f72351d1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 153 COLUMNS
At line 877 RHS
At line 1026 BOUNDS
At line 1156 ENDATA
Problem MODEL has 148 rows, 129 columns and 464 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 9 - 0.00 seconds
Cgl0004I processed model has 148 rows, 129 columns (129 integer (112 of which binary)) and 464 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0045I 1 integer variables out of 129 objects (129 integer) have cost

In [53]:
LpStatus[status]

'Optimal'

In [54]:
print(f"value of L: {value(L)}")
for i in range(n):
    print(value(pos_x[i]), value(pos_y[i]))

value of L: 12.0
6.0 6.0
12.0 8.0
9.0 7.0
6.0 0.0
9.0 0.0
12.0 0.0
3.0 0.0
3.0 9.0
